In [1]:

import torch
import pandas as pd
import dask.dataframe as dd
import torch
import multiprocessing as mp
import numpy as np
import csv
def read_csv(file_name):
    # Use pandas to read the CSV file
    data = pd.read_csv(file_name)
    return data

def convert_to_dict_v1(data):
    '''Convert DataFrame to list of dictionaries with keys: uid, word_vector, sum_vector, avg_vector.'''
    eval_locals = {'tensor': torch.tensor, 'torch': torch}
 
    list_data_dict = [{"uid": row[0], 
                       "word_vector": eval(row[1], eval_locals), 
                       "sum_vector": eval(row[2], eval_locals), 
                       "avg_vector": eval(row[3], eval_locals)} 
                      for row in data.itertuples(index=False)]
    
    return list_data_dict



In [2]:
# read json file and convert to dict
import json

def read_data(readPath):
    '''read csv file and return list of json objects.'''
    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)

    return taskData


## calculate cosine 

In [3]:
import numpy as np
import torch
import sys
import os
import multiprocessing as mp

from tqdm import tqdm
# from sklearn.metrics.pairwise import cosine_similarity
sys.path.append('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM')
from mlm_utils.model_utils import TOKENIZER


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-09 14:53:08.353395: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-09 14:53:09.037656: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 14:53:11.729931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-09 14:53:14.301203: E tensorflow/compiler/xla/stream_executor/cuda/cu

In [1]:
def cosine_module_2_tensors(tensor1, tensor2, cosine_func):

    norm_tensor1 = torch.norm(tensor1)
    norm_tensor2 = torch.norm(tensor2)
    
    cosine_sim_v2 = cosine_func(tensor1 / norm_tensor1, tensor2 / norm_tensor2)
    module_similarity = 1 - (torch.abs(norm_tensor1 - norm_tensor2) / (norm_tensor1 + norm_tensor2))
    
    return module_similarity * cosine_sim_v2


In [2]:
def cosine_sen_content_word(sum_tensor, avg_tensor, content_word_dict, cosine_func):
    '''một từ với 1 content word
    trả ra consine similarity giữa sum_vector_tensor và sum_vector của content word
    '''
   
    content_tensor_sum = content_word_dict['sum_vector'].clone().detach()
    content_tensor_avg = content_word_dict['avg_vector'].clone().detach()
    
    cosine_sum = cosine_func(sum_tensor/torch.norm(sum_tensor), content_tensor_sum/torch.norm(content_tensor_sum)).item()
    cosine_module_sum = cosine_module_2_tensors(sum_tensor, content_tensor_sum, cosine_func)
    
    cosine_avg = cosine_func(avg_tensor/torch.norm(avg_tensor), content_tensor_avg/torch.norm(content_tensor_avg)).item()
    cosine_module_avg = cosine_module_2_tensors(avg_tensor, content_tensor_avg, cosine_func)
    
    return cosine_sum, cosine_module_sum, content_word_dict['word_vector'], cosine_avg, cosine_module_avg

In [ ]:
def cosine_similarity(tensor1, tensor2):
    dot_product = torch.dot(tensor1, tensor2)
    magnitude1 = torch.norm(tensor1)
    magnitude2 = torch.norm(tensor2)
    return dot_product / (magnitude1 * magnitude2)

In [3]:
def cosine_sim_sen_list_word(sum_tensor, avg_tensor, list_dict_content_word):
    '''tinh cosine trong 1 cau voi tat ca cac content word. trả ra từ có cosine gần -1 và từ có cosine gần 0'''
    # list_result = list(map(lambda x: cosine_sen_content_word(sum_tensor, avg_tensor, x, cosine_func), list_dict_content_word))
    
    # 1 cau voi 1 list content word 
    min_similarity, replace_word_neg_cos_sum = min(((cosine_similarity(sum_tensor, list_dict_content_word[i]["sum_vector"]), list_dict_content_word[i]['word_vector']) for i in range(len(list_dict_content_word))), key=lambda x: x[0])
    return min_similarity, replace_word_neg_cos_sum


In [4]:
def replace_word_in_sentence(predicate_data_chunk, chunkNumber, tempList, content_word_data):
    ''' 1 câu gốc thành 2 câu mới: cosine = -1 và cosine = 0
    '''
    name = 'new_data_{}.json'.format(str(chunkNumber))

    cosine_func = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    def new_word_in_sen(predicate_data):
        masked_index = torch.where(torch.tensor(predicate_data['pos_tag_id']).clone().detach() != 0)

        predicate_np = np.array(predicate_data['origin_id'])
        
        replace_word_neg_cos = None
        replace_word_0_cos = None
        replace_word_neg_cos_module = None
        replace_word_0_cos_module = None
        
        avg_tensor = torch.tensor(predicate_data['avg_vector']).clone().detach()
        sum_tensor = torch.tensor(predicate_data['sum_vector']).clone().detach()
        if predicate_data['pos_tag_id'][masked_index[0][0].item()] == 1:
            min_cosin, replace_word = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word_data['noun'])
            
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 2:
            min_cosin, replace_word = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word_data['verb'], cosine_func)
            
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 3:
            min_cosin, replace_word = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word_data['adj'], cosine_func)
            
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 4:
            min_cosin, replace_word = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word_data['adv'], cosine_func)
            
        else: 
            KeyError('pos_tag_id not in [1, 2, 3, 4]')

        print("min cosine", min_cosin)
        print("replace word", replace_word)

        # cosine = -1
        origin_word = TOKENIZER.decode(predicate_np[masked_index[0]])
        # origin_sen = TOKENIZER.decode(predicate_np, skip_special_tokens = True)
        # cos_neg_sen_sum = origin_sen.replace(origin_word, TOKENIZER.decode(replace_word_cosine_neg_sum), 1) 
        cos_neg = TOKENIZER.decode(replace_word_neg_cos)
        cos_module_neg = TOKENIZER.decode(replace_word_neg_cos_module)
        
        # cosine = 0
        cos_0 = TOKENIZER.decode(replace_word_0_cos)
        cos_module_0 = TOKENIZER.decode(replace_word_0_cos_module)
        
        feature =  {
                "origin_uid": predicate_data['origin_uid'],
                "origin_id": predicate_data['origin_id'], 
                "masked_index": masked_index[0].tolist(),
                "masked_word": origin_word,
                "cos_neg": cos_neg, 
                "cos_0": cos_0, 
                "cos_module_neg": cos_module_neg,
                "cos_module_0": cos_module_0,
                "pos_tag_id": predicate_data['pos_tag_id']}
        return feature         
                
    list_feature = map(lambda x: new_word_in_sen(x), tqdm(predicate_data_chunk))
               
    with open(name, 'w') as wf:
        for feature in list_feature:
            wf.write('{}\n'.format(json.dumps(feature))) 
        tempList.append(name)

In [5]:
def create_new_data(predicate_file, dict_content_word, wriDir):
    '''
    return 4 data cho moi file:
        sum_vector va cosine = -1: uid, origin_input_id, sum_neg_input_id, pos_tag_id
        sum_vector va cosine = 0: uid, origin_input_id, sum_0_input_id, pos_tag_id
        avg_vector va cosine = -1: uid, origin_input_id, avg_neg_input_id, pos_tag_id
        avg_vector va cosine = 0: uid, origin_input_id, avg_0_input_id, pos_tag_id
    
    '''
    print("Preprocessing file... ", predicate_file)
    predicates_data = read_data(predicate_file)
    predicates_data = [item for sublist in predicates_data for item in sublist][:10]
    
    # MULTI PROCESSING
    man = mp.Manager()

    # shared list to store all temp files written by processes
    tempFilesList = man.list()
    
    # numProcess = mp.cpu_count() - 1
    numProcess = 1
    
    
    chunkSize = int(len(predicates_data) / (numProcess))
    print('Data Size: ', len(predicates_data))
    print('number of threads: ', numProcess)

    processes = []
    for i in range(numProcess):
        dataChunk = predicates_data[chunkSize*i : chunkSize*(i+1)]

        p = mp.Process(target = replace_word_in_sentence, args = (dataChunk, i, tempFilesList, dict_content_word))
        
        p.start()
        processes.append(p)
        
    for pr in processes:
        pr.join()
    
    wrtPath = wriDir + '{}'.format(predicate_file.split('/')[-1].replace('mlm_', ''))
    
    # combining the files written by multiple processes into a single final file
    with open(wrtPath, 'w') as f:
        for file in tempFilesList:
            with open(file, 'r') as r:
                for line in r:
                    sample =  json.loads(line)
                    f.write('{}\n'.format(json.dumps(sample)))
            os.remove(file)
        
    print("Done file", predicate_file)   


In [9]:
import time
import dask.dataframe as dd
def read_and_convert_csv(file_path):
    '''Read a CSV file and convert it to a dictionary.'''
    data = read_csv(file_path)
    return convert_to_dict_v1(data)

file_paths = {
    "noun": "./list_content_word/NOUN.csv",
    "verb": "./list_content_word/VERB.csv",
    "adj": "./list_content_word/ADJ.csv",
    "adv": "./list_content_word/ADV.csv"
}


dict_content_word = {key: read_and_convert_csv(file_path) for key, file_path in file_paths.items()}


In [ ]:
from mlm_utils.transform_func import get_files
# wriDir = './pertured_data/'
wriDir = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/process_folder'
dataDir = './word_present_each_file/'
files = get_files(dataDir)

# for file in files:
    
create_new_data(dataDir + files[0], dict_content_word, wriDir)

In [8]:
l = [101, 170, 176, 118, 1106, 118, 170, 6468, 1120, 1103, 1148, 183, 21977, 26918, 23767, 1104, 27553, 1179, 123, 1104, 5351, 122, 8632, 2999, 188, 1643, 22548, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(l)

128